<a href="https://colab.research.google.com/github/NickAI2811/DataToolKit/blob/main/Gemma_RAG_ChromaDB_GenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Using Gemma without RAG


### Step 1: Install libraries

In [ ]:
pip install langchain-classic --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.1/476.1 kB 30.6 MB/s eta 0:00:00


In [ ]:
!pip install -q langchain transformers torch --quiet
!pip install langchain-community --quiet
!pip install langchain_huggingface --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


### Step 2: Import Packages

In [ ]:
#from langchain_community.llms import HuggingFacePipeline
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFaceEndpoint
from huggingface_hub import notebook_login
import torch
import os
# from google.colab import userdata

In [ ]:
# Set up the Hugging Face Hub API token and login to huggingface_hub
from huggingface_hub import login
login()

In [ ]:
# Configuration
MODEL_NAME = "google/gemma-3-1b-it"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    dtype=torch.float16 if DEVICE == "cuda" else torch.float32
)

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [ ]:
# Create text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1000,
    do_sample=True,
    temperature=0.7
)

Device set to use cuda:0


In [ ]:
# Wrap in LangChain
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
user_input = input("\nYou: ")

# Format prompt for Gemma
prompt = f"<start_of_turn>user\n{user_input}<end_of_turn>\n<start_of_turn>model\n"

# Generate response
response = llm.invoke(prompt)

# Extract just the model's response
if isinstance(response, list):
    response = response[0]['generated_text']

# Clean up the output
response = response.split("<start_of_turn>model\n")[-1].strip()
print(f"\nAssistant: {response}")


You: How gender is determined?

Assistant: Okay, let's dive into the incredibly complex and fascinating topic of how gender is determined. It's a question that scientists, philosophers, and psychologists have been grappling with for centuries, and there's no single, universally accepted answer. Here's a breakdown of the current understanding, broken down into key areas:

**1. Biological Factors – The Genetic Component:**

* **Chromosomes:** The most significant biological influence comes from our chromosomes.  Humans typically have two X chromosomes (XX) and one X and one Y chromosome (XY).
* **Sex Chromosomes:** The Y chromosome is generally considered to be solely involved in male development.  The X chromosome carries most of the genes related to female characteristics.
* **Genes Involved:** Research has identified numerous genes involved in sex determination, including:
    * **SRY gene:** This is the most crucial gene identified as directly influencing male development. It's trig

## Using Gemma with RAG

### Step 0: Install additional packages

In [ ]:
!pip install -q -U chromadb pypdf sentence-transformers accelerate --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.2/328.2 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.8 MB/s et

In [ ]:
!pip install pymupdf --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 67.1 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_classic.chains import RetrievalQA
# from langchain.chains import RetrievalQA
from langchain_classic.prompts import PromptTemplate
from langchain_classic.docstore.document import Document

In [ ]:
# ======================
# CONFIGURATION
# ======================
PDF_FOLDER_PATH = "./Book_Class_10_Ch8/"  # Folder containing your PDFs
EMBEDDING_MODEL = "sentence-transformers/all-mpnet-base-v2"

In [ ]:
os.makedirs(PDF_FOLDER_PATH, exist_ok=True)

### Step 1: Data Preparation

*   Read text from PDF documents
*   Divide PDF document into smaller chunks  

In [ ]:
# import pymupdf as fitz

In [ ]:
import fitz

In [ ]:
def load_and_chunk_pdfs(pdf_folder):
    documents = []

    for pdf_file in os.listdir(pdf_folder):
        if pdf_file.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, pdf_file)
            print(f"Processing {pdf_file}...")

            # Load PDF with PyMuPDF
            doc = fitz.open(pdf_path)
            print("loading done")

            pages = []
            for page_number in range(len(doc)):
                page = doc[page_number]
                text = page.get_text("text")

                # Create Document object per page (similar to PyPDFLoader behavior)
                pages.append(
                    Document(
                        page_content=text,
                        metadata={"source": pdf_path, "page": page_number + 1}
                    )
                )

            print("pages loaded")

            # Split into chunks
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200,
                length_function=len
            )

            chunks = text_splitter.split_documents(pages)
            documents.extend(chunks)

    print(f"\nTotal documents: {len(documents)}")
    return documents

### Step 2: Create VectorStore

In [ ]:
# ======================
# CREATE VECTORSTORE
# ======================
def create_vectorstore(documents):
    # Initialize embedding function
    embedding_function = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL,
        model_kwargs={"device": DEVICE},
    )

    # Create ChromaDB
    vector_db = Chroma.from_documents(
        documents=documents,
        embedding=embedding_function
    )
    return vector_db


### Step 3: Setup Gemma Q&A Chain


*   Get access tokens from Hugging face
*   Initialize the tokenizer with the model.
*   Create a text generation pipeline.
*   Initialize the LLM with pipeline and model arguments
*   The final step is to generate the answers using both the vector store and the LLM.

It will generate embeddings to the input query or question retrieve the context from the vector store, and feed this to the LLM to generate the answers

In [ ]:
# ======================
# SETUP GEMMA QA CHAIN
# ======================
def setup_qa_chain(vector_db):
    # Load Gemma model
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32,
    )


    # Create pipeline
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
    )

    # Wrap in LangChain
    llm = HuggingFacePipeline(
        pipeline=pipe,
        model_kwargs={
            "max_new_tokens": 512,
            "do_sample": True,
            "temperature": 0.7,
            "top_k": 50,
            "top_p": 0.95,
        }
    )

    # Custom prompt for Gemma
    prompt_template = """<start_of_turn>user
    Use the following context to answer the question. Be concise and accurate.
    If you don't know, say you don't know. Don't make up answers.

    Context: {context}

    Question: {question}<end_of_turn>
    <start_of_turn>model
    """

    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )

    # Create QA chain
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vector_db.as_retriever(search_kwargs={"k": 3}),
        chain_type_kwargs={"prompt": PROMPT},
        return_source_documents=True
    )

    return qa_chain

### Step 4: Main execution

In [ ]:
# ======================
# MAIN EXECUTION
# ======================

# Step 1: Process PDFs
print("Loading and chunking PDFs...")
documents = load_and_chunk_pdfs(PDF_FOLDER_PATH)

# Step 2: Create vector store
print("\nCreating vector store...")
vector_db = create_vectorstore(documents)

# Step 3: Setup QA system
print("\nSetting up QA system...")
qa_chain = setup_qa_chain(vector_db)


Loading and chunking PDFs...
Processing jesc108.pdf...
loading done
pages loaded

Total documents: 21

Creating vector store...


/tmp/ipython-input-3985321207.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


Setting up QA system...


`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0


In [ ]:
# question answering

question = input("\nYour question: ")
result = qa_chain.invoke({"query": question})
response = result["result"].split("<start_of_turn>model\n")[-1].strip()
print("\nAnswer:", response)

# print("\nAnswer:", result["result"])



# Show sources
print("\nSources:")
for i, doc in enumerate(result["source_documents"], 1):
    print(f"\nSource {i}:")
    print("Content:", doc.page_content[:200] + "...")
    print("From:", doc.metadata.get("source", "Unknown"))


Your question: How gender is determined?

Answer: According to the text, gender is determined by what the child inherits from their father.

Sources:

Source 1:
Content: from their mother regardless of whether they are boys or
girls. Thus, the sex of the children will be determined by
what they inherit from their father. A child who inherits
an X chromosome from her f...
From: ./Book_Class_10_Ch8/jesc108.pdf

Source 2:
Content: inheritance might work?
8.2.4 Sex Determination
We have discussed the idea that the two sexes participating in sexual
reproduction must be somewhat different from each other for a number
of reasons. H...
From: ./Book_Class_10_Ch8/jesc108.pdf

Source 3:
Content: But so far, we have assumed that similar gene sets are
inherited from both parents. If that is the case, how can
genetic inheritance determine sex?
The explanation lies in the fact that all human
chro...
From: ./Book_Class_10_Ch8/jesc108.pdf
